In [8]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
import sys, os
sys.path.append(os.path.realpath('..'))

from mgc.independence import *
from mgc.sims import linear

In [9]:
N = [
    50,
    100,
    200,
    500,
    1000,
    2000,
    5000,
    10000
]

tests = [
    Dcorr,
    MGC,
]

In [10]:
for test in tests:
    times = []
    for n in N:
        x, y = linear(n, 1, noise=True)
        time = %timeit -n 1 -r 5 -q -o test().test(x, y, workers=-1)
        times.append(np.mean(time.timings))
    np.savetxt('../benchmarks/perf/{}.csv'.format(test.__name__), times, delimiter=',')

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[3:])

In [ ]:
FONTSIZE = 30

def plot_wall_times():
    fig = plt.figure(figsize=(20,15))
    
    custom_color = {
        "Dcorr" : "#377eb8",
        "MGC" : "#e41a1c",
    }
    
    for test in tests:
        test_times = np.genfromtxt('../benchmarks/2samp_perf/{}.csv'.format(test.__name__), delimiter=',')
        if test.__name__ in custom_color.keys():
            plt.plot(N, test_times, custom_color[test_name], label=test.__name__, lw=5)
        else:
            plt.plot(N, test_times, label=test.__name__, lw=5)
    
    leg = plt.legend(bbox_to_anchor=(1.17, 0.45), bbox_transform=plt.gcf().transFigure,
                     ncol=2, loc='upper center', fontsize=FONTSIZE)
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.savefig('../benchmarks/figs/2samp_wall_times.pdf', transparent=True, bbox_inches='tight')

In [ ]:
plot_wall_times()